In [30]:
import os
import sys
import datetime
import re
import pandas as pd

In [35]:
def file_merger(full_path):
        """
        Method that gets all the saved files that contain data from *Mercados y precios*, as well as
        *Generación y consumo* and combines them into a single .csv file that will form the complete dataset.
        """
        
        # Get all filenames with .csv extension in data folder
        filenames = [filename for filename in os.listdir(full_path) if filename.endswith(".csv") and "renewable_generation" in filename]
        
        # Create a list of dataframes from the files
        dfs = [pd.read_csv(os.path.join(full_path, filename), sep=",") for filename in filenames]
        
        # Concatenate all dataframes by rows into one
        df = pd.concat(dfs, ignore_index=True, axis=0)
        
        # Save the final dataframe to a .csv file
        df.to_csv(os.path.join(full_path, "dataset.csv"), sep=";", index=False)
        print("Data has been merged and saved to data/dataset.csv")

In [39]:
def create_categorical_column(full_path):
        """
        Method that transforms the dataset by creating a categorical column from the renewable 
        energy column names (energy_source) and another column with the values.
        
        Saves the newly generated dataframe to a new csv file.
        """

        df = pd.read_csv(os.path.join(full_path, "dataset.csv"), sep=";")
        
        dataset_dict = {
            'date': [], 'hour': [], 'avg total price (euro/MWh)': [], 'avg price free market (euro/MWh)': [], 'avg price reference market (euro/MWh)': [], 
            'energy total (MWh)': [], 'energy free market (MWh)': [], 'energy reference market (MWh)': [], 'free market share (%)': [], 
            'reference market share (%)': [], 'renewable generation (%)': [], 'energy_source': [], 'renewable generation (MW)': []}

        keep_cols = ['date', 'hour', 'avg total price (euro/MWh)', 'avg price free market (euro/MWh)', 
                                'avg price reference market (euro/MWh)', 'energy total (MWh)', 'energy free market (MWh)', 
                                'energy reference market (MWh)', 'free market share (%)', 'reference market share (%)', 'renewable generation (%)']

        # Transform the dataframe to the final shape
        # Add a new row for each renewable energy source and the corresponding values for each renewable energy column
        for idx, row in df.iterrows():
            for col in df.columns:
                if col in ['renewable generation (MW)','wind generation (MW)', 'water generation (MW)', 
                            'solar generation (MW)', 'nuclear generation (MW)', 'thermorenewable generation (MW)']:
                    for keep_col in keep_cols:
                        dataset_dict[keep_col].append(row[keep_col])
                    dataset_dict['energy_source'].append(col)
                    dataset_dict['renewable generation (MW)'].append(row[col])

        dataset_dict_df = pd.DataFrame(dataset_dict)
        dataset_dict_df.to_csv(os.path.join(full_path, "final_dataset.csv"), sep=";", index=False)
        print("Data has been transformed and saved to data/final_dataset.csv")

In [40]:
# Create list with all files in the save directory
path = "/home/albert/Desktop/Tipologia i cicle de vida de les dades/PRA1/"
full_path = os.path.join(path, 'data', 'already_merged')
files = os.listdir(full_path)

In [41]:
create_categorical_column(full_path)

Data has been transformed and saved to data/final_dataset.csv


In [47]:
# get the final_dataset.csv file    
final_dataset = pd.read_csv(os.path.join(full_path, "final_dataset.csv"), sep=";")
    
# transform the date column from 1-5-2021 format to 2021-05-01
final_dataset['date'] = pd.to_datetime(final_dataset['date'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')

In [49]:
final_dataset.to_csv(os.path.join(full_path, "final_dataset.csv"), sep=";", index=False)

In [48]:
final_dataset

,date,hour,avg total price (euro/MWh),avg price free market (euro/MWh),avg price reference market (euro/MWh),energy total (MWh),energy free market (MWh),energy reference market (MWh),free market share (%),reference market share (%),renewable generation (%),energy_source,renewable generation (MW)
0,2022-01-12,00:00,"199,18","199,18","199,19","26.714,2","23.764,1","2.950,1","89,0","11,0","68,6\n%",renewable generation (MW),20.548MW
1,2022-01-12,00:00,"199,18","199,18","199,19","26.714,2","23.764,1","2.950,1","89,0","11,0","68,6\n%",wind generation (MW),11.926MW
2,2022-01-12,00:00,"199,18","199,18","199,19","26.714,2","23.764,1","2.950,1","89,0","11,0","68,6\n%",water generation (MW),998MW
3,2022-01-12,00:00,"199,18","199,18","199,19","26.714,2","23.764,1","2.950,1","89,0","11,0","68,6\n%",solar generation (MW),27MW
4,2022-01-12,00:00,"199,18","199,18","199,19","26.714,2","23.764,1","2.950,1","89,0","11,0","68,6\n%",nuclear generation (MW),6.994MW
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105115,2021-10-01,23:00,"222,18","222,17","222,21","25.044,7","22.535,5","2.509,2","90,0","10,0","49,7\n%",wind generation (MW),2.380MW
105116,2021-10-01,23:00,"222,18","222,17","222,21","25.044,7","22.535,5","2.509,2","90,0","10,0","49,7\n%",water generation (MW),2.870MW
105117,2021-10-01,23:00,"222,18","222,17","222,21","25.044,7","22.535,5","2.509,2","90,0","10,0","49,7\n%",solar generation (MW),19MW
105118,2021-10-01,23:00,"222,18","222,17","222,21","25.044,7","22.535,5","2.509,2","90,0","10,0","49,7\n%",nuclear generation (MW),6.931MW


In [24]:
# energy_prices_dict = {
#     'date': [], 'hour': [], 'avg total price (euro/MWh)': [], 'avg price free market (euro/MWh)': [], 'avg price reference market (euro/MWh)': [], 
#     'energy total (MWh)': [], 'energy free market (MWh)': [], 'energy reference market (MWh)': [], 'free market share (%)': [], 'reference market share (%)': [],
#     'renewable generation (%)': [], 'energy_source': [], 'renewable generation (MW)': []}

# energy_prices_cols = ['date', 'hour', 'avg total price (euro/MWh)', 'avg price free market (euro/MWh)', 
#                         'avg price reference market (euro/MWh)', 'energy total (MWh)', 'energy free market (MWh)', 
#                         'energy reference market (MWh)', 'free market share (%)', 'reference market share (%)', 
#                         'renewable generation (%)']

# for idx, row in energy_prices.iterrows():
#     for col in energy_prices.columns:
#         if col in ['renewable generation (MW)','wind generation (MW)', 'water generation (MW)', 
#                     'solar generation (MW)', 'nuclear generation (MW)', 'thermorenewable generation (MW)']:
#             for energy_col in energy_prices_cols:
#                 energy_prices_dict[energy_col].append(row[energy_col])
#             energy_prices_dict['energy_source'].append(col)
#             energy_prices_dict['renewable generation (MW)'].append(row[col])

# energy_prices_dict_df = pd.DataFrame(energy_prices_dict)

In [25]:
# # save df to csv
# energy_prices_dict_df.to_csv(os.path.join(full_path, "test.csv"), index=False)

In [26]:
# # Print the length of the different keys
# for key in energy_prices_dict.keys():
#     print(f"{key}: {len(energy_prices_dict[key])}")

date: 144
hour: 144
avg total price (euro/MWh): 144
avg price free market (euro/MWh): 144
avg price reference market (euro/MWh): 144
energy total (MWh): 144
energy free market (MWh): 144
energy reference market (MWh): 144
free market share (%): 144
reference market share (%): 144
renewable generation (%): 144
energy_source: 144
renewable generation (MW): 144
